In [1]:
from scrampy import splice
import pandas as pd
from pydub.audio_segment import AudioSegment

Read in Blueprint
-----------------

In [2]:
bp = pd.read_csv('simple/interl+gaps_blueprint.csv', index_col=0)
bp

,name,start,end,length,order
0,NTF,0,8921,8921,0
1,gap,0,1579,1579,-1
2,pieman,14000,26970,12970,0
3,gap,0,2030,2030,-1
4,NTF,8921,19694,10773,1
5,gap,0,1227,1227,-1
6,pieman,26970,41620,14650,1
7,gap,0,1850,1850,-1


Load Audio and Use Blueprint to Interleave
---------------------------------------------

In [3]:
aud_paths = {
    'NTF':    'aud/NTF/NotTheFall.mp3',
    'pieman': 'aud/pieman/pieman.mp3',
    'gap':    'aud/silence.mp3'
}

aud_files = {k: AudioSegment.from_mp3(v) for k,v in aud_paths.items()}

interl = splice.aud_from_log(bp, **aud_files)

Examine Interleaved Audio
-------------------------

In [4]:
# length in milliseconds
print len(interl)

# decrease volume slightly
shh_interl = interl - 1

# Summaries of amplitude
print interl.rms, ',', interl.dBFS
print shh_interl.rms, ',', shh_interl.dBFS

54000
1571 , -26.3854749984
1400 , -27.3864379856


Pull out Pieman Segments
------------------------

In [22]:
# just the pieman segments
bp_pie = bp[bp['name'] == 'pieman']
aud_pie = aud_files['pieman']

# create blueprint for interleaved audio
bp_interl = bp.copy()
splice.update_splits(bp_interl)
bp_interl_pie = splice.align_blueprints(bp_interl, bp_pie, src_on=('name', 'order'))

# get pieman clip from original
start, end = bp_pie[['start', 'end']].iloc[0]
orig_clip = aud_pie[start:end]

# get from interleaved
start, end = bp_interl_pie[['start', 'end']].iloc[0]
new_clip  = interl[start:end]

# same length
len(orig_clip) == len(new_clip)

True

Play Each Clip
--------------

In [12]:
from pydub.playback import play
#play(orig_clip)
#play(new_clip)